In [1]:
import pandas as pd
import numpy as np

In [2]:
gals = pd.read_csv('data/planck1/henriques2015a_z6p42_mstar.csv', skiprows=101, skipfooter=1)

/home/chris/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [3]:
gals.shape

(29175, 21)

In [5]:
import hightolowz

In [16]:
dimensions = np.array([500, 500, 500])

In [3]:
'data/wmap1/dgal_mstar9_r%s_%s.csv' % ('this','random')

'data/wmap1/dgal_mstar9_rthis_random.csv'

In [5]:
gals

,zn_galaxyId,zn_haloId,zn_fofCentralId,zn_phKey,zn_x,zn_y,zn_z,zn_stellarMass,zn_sfr,zn_vvir,...,zn_blackHoleMass,zn_xrayLum,zn_quasarAccretionRate,zn_radioAccretionRate,zn_coldGas,zn_hotGas,zn_metalsHotGas,z0_haloId,z0_centralId,z0_central_mcrit200
0,10000036106924,10000036049629,10000036106924,16440270,32.052933,82.328470,249.038680,0.225126,14.205048,206.41011,...,0.000000,44.993702,0.000000,0.000000e+00,0.716967,1.339435,0.008263,1.000004e+13,1.000004e+13,22817.58200
1,10000036110040,10000036053478,10000036110040,16439819,31.516111,85.095520,252.991970,0.135219,20.631071,203.04420,...,0.000000,45.090767,0.000000,0.000000e+00,0.351238,1.654535,0.006034,1.000004e+13,1.000004e+13,22817.58200
2,8012934039610,8012934026973,8012934039610,2676102,31.244972,474.671100,175.104830,0.119678,18.825102,194.30869,...,0.000000,44.967190,0.000000,0.000000e+00,0.243331,1.353740,0.005196,8.012934e+12,8.012934e+12,41718.63000
3,4000099000047,4000099000047,4000099000047,913474,122.415245,56.816116,82.520080,0.189437,24.894884,231.84580,...,0.000000,45.351810,0.000000,0.000000e+00,0.730402,2.737513,0.008158,4.000099e+12,4.000099e+12,547.65530
4,20000025000047,20000025000047,20000025000047,659532,183.813370,108.284996,148.146220,0.125645,11.667884,200.73674,...,0.000000,44.928238,0.000000,0.000000e+00,0.565432,1.299969,0.003791,2.000003e+13,2.000003e+13,1113.21470
5,39009266000047,39009266000047,39009266000047,1298055,188.263370,224.198880,110.570160,0.121839,16.688196,183.31909,...,0.000000,44.775830,0.000000,0.000000e+00,0.214888,0.978841,0.005500,3.900927e+13,3.900927e+13,4339.10940
6,39009454000047,39009454000047,39009454000047,1291220,190.201920,229.401960,66.856740,0.113427,17.322300,190.49765,...,0.000000,44.922960,0.000000,0.000000e+00,0.214888,1.271049,0.005032,3.900945e+13,3.900945e+13,298.26877
7,58000208000047,58000208000047,58000208000047,950203,121.514946,117.119950,70.955360,0.208256,31.298580,226.16684,...,0.000000,45.322052,0.000000,0.000000e+00,0.521914,2.403754,0.009652,5.800021e+13,5.800021e+13,378.13528
8,51018313000047,51018313000047,51018313000047,1712696,57.919770,150.602430,129.178910,0.164068,19.303799,198.77238,...,0.000000,44.979794,0.000000,0.000000e+00,0.385267,1.333048,0.006587,5.101831e+13,5.101831e+13,192.38681
9,52000039001470,52000039001442,52000039001470,1718727,32.576366,150.284880,157.670850,0.278026,37.314407,248.42180,...,0.000000,45.448956,0.000000,0.000000e+00,0.634759,3.366188,0.013506,5.200004e+13,5.200004e+13,8975.02700


In [8]:
gals.ix[np.isnan(gals['z0_haloId']), 'z0_haloId'] = -1

In [9]:
gals

,zn_galaxyId,zn_haloId,zn_fofCentralId,zn_phKey,zn_x,zn_y,zn_z,zn_stellarMass,zn_sfr,zn_vvir,...,zn_blackHoleMass,zn_xrayLum,zn_quasarAccretionRate,zn_radioAccretionRate,zn_coldGas,zn_hotGas,zn_metalsHotGas,z0_haloId,z0_centralId,z0_central_mcrit200
0,10000036106924,10000036049629,10000036106924,16440270,32.052933,82.328470,249.038680,0.225126,14.205048,206.41011,...,0.000000,44.993702,0.000000,0.000000e+00,0.716967,1.339435,0.008263,1.000004e+13,1.000004e+13,22817.58200
1,10000036110040,10000036053478,10000036110040,16439819,31.516111,85.095520,252.991970,0.135219,20.631071,203.04420,...,0.000000,45.090767,0.000000,0.000000e+00,0.351238,1.654535,0.006034,1.000004e+13,1.000004e+13,22817.58200
2,8012934039610,8012934026973,8012934039610,2676102,31.244972,474.671100,175.104830,0.119678,18.825102,194.30869,...,0.000000,44.967190,0.000000,0.000000e+00,0.243331,1.353740,0.005196,8.012934e+12,8.012934e+12,41718.63000
3,4000099000047,4000099000047,4000099000047,913474,122.415245,56.816116,82.520080,0.189437,24.894884,231.84580,...,0.000000,45.351810,0.000000,0.000000e+00,0.730402,2.737513,0.008158,4.000099e+12,4.000099e+12,547.65530
4,20000025000047,20000025000047,20000025000047,659532,183.813370,108.284996,148.146220,0.125645,11.667884,200.73674,...,0.000000,44.928238,0.000000,0.000000e+00,0.565432,1.299969,0.003791,2.000003e+13,2.000003e+13,1113.21470
5,39009266000047,39009266000047,39009266000047,1298055,188.263370,224.198880,110.570160,0.121839,16.688196,183.31909,...,0.000000,44.775830,0.000000,0.000000e+00,0.214888,0.978841,0.005500,3.900927e+13,3.900927e+13,4339.10940
6,39009454000047,39009454000047,39009454000047,1291220,190.201920,229.401960,66.856740,0.113427,17.322300,190.49765,...,0.000000,44.922960,0.000000,0.000000e+00,0.214888,1.271049,0.005032,3.900945e+13,3.900945e+13,298.26877
7,58000208000047,58000208000047,58000208000047,950203,121.514946,117.119950,70.955360,0.208256,31.298580,226.16684,...,0.000000,45.322052,0.000000,0.000000e+00,0.521914,2.403754,0.009652,5.800021e+13,5.800021e+13,378.13528
8,51018313000047,51018313000047,51018313000047,1712696,57.919770,150.602430,129.178910,0.164068,19.303799,198.77238,...,0.000000,44.979794,0.000000,0.000000e+00,0.385267,1.333048,0.006587,5.101831e+13,5.101831e+13,192.38681
9,52000039001470,52000039001442,52000039001470,1718727,32.576366,150.284880,157.670850,0.278026,37.314407,248.42180,...,0.000000,45.448956,0.000000,0.000000e+00,0.634759,3.366188,0.013506,5.200004e+13,5.200004e+13,8975.02700
